In [1]:
import pybamm as pb;
import matplotlib as mpl; 
font = {'family' : 'DejaVu Sans','size'   : 15}
mpl.rc('font', **font)

In [3]:
model = pb.lithium_ion.DFN()
model_2 = pb.lithium_ion.DFN(options={
    "thermal": "lumped",
    "particle": "Fickian diffusion",          
    "SEI":"interstitial-diffusion limited",   
    "SEI on cracks":"true",  
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",      
    "particle mechanics":("swelling and cracking", "swelling only"), 
    "loss of active material":"stress-driven", 
    "lithium plating":"partially reversible"
    },)
param = pb.ParameterValues("OKane2023")
# Adjust SEI parameters
param.update({"Inner SEI lithium interstitial diffusivity [m2.s-1]": 5e-19})
# Override SEI thickness dependence of lithium plating
param.update({"Dead lithium decay rate [s-1]": 1e-6})
param.update({"Lithium plating kinetic rate constant [m.s-1]": 1e-10})
# Adjust mechanical degradation parameters
param.update({"Negative electrode cracking rate": 1e-22})
param.update({"Negative electrode LAM constant proportional term [s-1]": 5e-8})
param.update({"Positive electrode LAM constant proportional term [s-1]": 1e-8})


In [5]:
# define experiment
V_max = 4.2;        V_min = 2.5; 
exp_RPT_text = [ ( 
    f"Charge at 0.3C until {V_max} V",  
    f"Hold at {V_max}V until C/100",
    "Rest for 1 hours (5 minute period)",  
    f"Discharge at 0.1C until {V_min} V (5 minute period)",  
    "Rest for 1 hours (5 minute period)",  
    f"Charge at 0.1C until {V_max} V (5 minute period)",
    f"Hold at {V_max} V until C/100",
    "Rest for 1 hours (5 minute period)",
    f"Discharge at 1C until {V_min}V", 
    "Rest for 3 hours (15 minute period)",
    ) ]
Experiment_Breakin= pb.Experiment( exp_RPT_text * 1     )


In [6]:
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 30,  # negative particle
    "r_p": 20,  # positive particle
}

In [7]:
sim1 = pb.Simulation(
    model, 
    experiment=Experiment_Breakin,
    parameter_values=param,
    solver=pb.CasadiSolver(),
    var_pts=var_pts,
)
solution1 = sim1.solve(
    #save_at_cycles=1
)

#stop = timeit.default_timer()
#print('running time: ' + str(stop - start) +'s')

In [8]:
sim2 = pb.Simulation(
    model_2, 
    experiment=Experiment_Breakin,
    parameter_values=param,
    solver=pb.CasadiSolver(),
    var_pts=var_pts,
)
solution2 = sim2.solve(
    #save_at_cycles=1
)

In [9]:
param["Negative electrode porosity"]
print(
    solution1['Negative electrode porosity'].entries[:,0],
    solution2['Negative electrode porosity'].entries[:,0],)

[0.25 0.25 0.25 0.25 0.25] [0.20891532 0.20891532 0.20891532 0.20891532 0.20891532]


In [10]:
print(
    solution1.cycles[0]['X-averaged negative electrode porosity'].entries[-1],
    solution1.cycles[0]['X-averaged negative electrode porosity'].entries[0],)

0.25 0.25


In [11]:
# plot SOH, LAM, LII
# SOH must be defined ourselves? 
print(solution1.cycles[0]['Loss of capacity to SEI [A.h]'].entries[-1])
print(solution2.cycles[-1]['Loss of capacity to SEI [A.h]'].entries[-1]-solution2.cycles[0]['Loss of capacity to SEI [A.h]'].entries[0])

0.0
0.0012225361408754604


In [14]:
label = ["Blank DFN","All ageing on"] 
var = [
    "Discharge capacity [A.h]",
    "Terminal voltage [V]",
    "Negative electrode porosity",
    "LAM_ne [%]",
    'LLI [%]', 
    'X-averaged negative electrode porosity'
]
quick_plot = pb.QuickPlot(
    [solution1,solution2], var,label,variable_limits='tight') #     
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=26.759424835105833, step=0.26759424835105833…